In [1]:
# Mount Google Drive (Because i am using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentence_transformers
!pip install sacremoses
import string 
import re
import nltk
from nltk import  word_tokenize, pos_tag, pos_tag_sents
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet') 
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sacremoses import MosesDetokenizer
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import torch
import scipy
import json
import pandas as pd
import gzip
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=68c58e86cb2162881ad8c6bceeea78e83e224dd1b045097a42d7fe89ec39f021
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
# Lets read our dataset using pandas
df = pd.read_csv('/content/drive/MyDrive/Anime_Recommender/animelist.csv')
df.head()

,MAL ID,Title,Medium Type,Original Source,Status,Rating,Score,ScoredBy,Members,Favourite,Synopsis,Producers,Studios,Genres,Themes,Demographic
0,51535,Shingeki no Kyojin: The Final Season - Kankets...,Special,Manga,Currently Airing,R - 17+ (violence & profanity),9.20,78537.0,327033,6361,The conclusion to Shingeki no Kyojin.,"['Production I.G', 'Dentsu', 'Mainichi Broadca...",['MAPPA'],"['Action', 'Drama', 'Suspense']","['Gore', 'Military', 'Survival']",['Shounen']
1,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,Finished Airing,R - 17+ (violence & profanity),9.11,1972869.0,3103521,214047,After a horrific alchemy experiment goes wrong...,"['Aniplex', 'Square Enix', 'Mainichi Broadcast...",['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy']",['Military'],['Shounen']
2,41467,Bleach: Sennen Kessen-hen,TV,Manga,Finished Airing,R - 17+ (violence & profanity),9.09,179829.0,399540,16156,Substitute Soul Reaper Ichigo Kurosaki spends ...,"['TV Tokyo', 'Aniplex', 'Dentsu', 'Shueisha']",['Pierrot'],"['Action', 'Adventure', 'Fantasy']",NaN,['Shounen']
3,9253,Steins;Gate,TV,Visual novel,Finished Airing,PG-13 - Teens 13 or older,9.08,1309377.0,2389026,180062,Eccentric scientist Rintarou Okabe has a never...,"['Frontier Works', 'Media Factory', 'Movic', '...",['White Fox'],"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",NaN
4,28977,Gintama°,TV,Manga,Finished Airing,PG-13 - Teens 13 or older,9.07,232370.0,580694,15592,"Gintoki, Shinpachi, and Kagura return as the f...","['TV Tokyo', 'Aniplex', 'Dentsu']",['Bandai Namco Pictures'],"['Action', 'Comedy', 'Sci-Fi']","['Gag Humor', 'Historical', 'Parody', 'Samurai']",['Shounen']


In [4]:
df.tail()


,MAL ID,Title,Medium Type,Original Source,Status,Rating,Score,ScoredBy,Members,Favourite,Synopsis,Producers,Studios,Genres,Themes,Demographic
22177,45187,Zuqiu Jianghu,ONA,Original,Finished Airing,G - All Ages,NaN,NaN,46,0,NaN,NaN,NaN,['Sports'],['Team Sports'],NaN
22178,51288,Zuoshou Shanglan,ONA,Original,Currently Airing,PG-13 - Teens 13 or older,NaN,NaN,5116,51,"The story follows teenager Xu Xingze, a passio...","['Tencent Penguin Pictures', 'LHL Culture']",NaN,['Sports'],['Team Sports'],NaN
22179,46436,Zuoyou Nao Zhili Kaifa,TV,Original,Finished Airing,PG - Children,NaN,NaN,31,0,NaN,NaN,NaN,['Fantasy'],NaN,['Kids']
22180,45069,Zuqiu Wangzhe,Movie,Original,Finished Airing,G - All Ages,NaN,NaN,52,0,NaN,NaN,NaN,"['Adventure', 'Fantasy', 'Sports']",NaN,['Kids']
22181,52512,Zurui Maboroshi,Music,Original,Finished Airing,PG-13 - Teens 13 or older,NaN,NaN,116,0,"Music video for the song Zurui Maboroshi, perf...",NaN,['StudioXD'],NaN,['Music'],NaN


In [5]:
# Lets see some other info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22182 entries, 0 to 22181
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MAL ID           22182 non-null  int64  
 1   Title            22182 non-null  object 
 2   Medium Type      22181 non-null  object 
 3   Original Source  22182 non-null  object 
 4   Status           22182 non-null  object 
 5   Rating           21835 non-null  object 
 6   Score            13812 non-null  float64
 7   ScoredBy         13812 non-null  float64
 8   Members          22182 non-null  int64  
 9   Favourite        22182 non-null  int64  
 10  Synopsis         18050 non-null  object 
 11  Producers        9914 non-null   object 
 12  Studios          12390 non-null  object 
 13  Genres           17555 non-null  object 
 14  Themes           12745 non-null  object 
 15  Demographic      9272 non-null   object 
dtypes: float64(2), int64(3), object(11)
memory usage: 2.7+ MB


In [6]:
# Lets do some initial cleaning to get rid of some values

# Lets remove all animes that scored below 6.5 and those that are blank as this would provide more quality anime for the user
anime=df.query('Score >= 1')

# Lets also get rid of all anime that are OVA,Special and Music as this is not relevant
anime = anime[anime['Medium Type'].isin(['TV','Movie','ONA'])]

# Lets also remove all the blank rows from synopsis as this is the most important field for my system
# drop rows with blank values in the 'synopsis' column
anime = anime.dropna(subset=['Synopsis'])

# Lets also remove the phrase '[Written by MAL Rewrite]' from synopsis as well
anime['Synopsis'] = anime['Synopsis'].str.replace(r'\[Written by MAL Rewrite\]', '', regex=True)
anime = anime

In [7]:
# find the longest length of the Synopsis column
max_length = anime['Synopsis'].str.len().min()
print(max_length)

14


In [8]:
anime.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8444 entries, 1 to 13810
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MAL ID           8444 non-null   int64  
 1   Title            8444 non-null   object 
 2   Medium Type      8444 non-null   object 
 3   Original Source  8444 non-null   object 
 4   Status           8444 non-null   object 
 5   Rating           8386 non-null   object 
 6   Score            8444 non-null   float64
 7   ScoredBy         8444 non-null   float64
 8   Members          8444 non-null   int64  
 9   Favourite        8444 non-null   int64  
 10  Synopsis         8444 non-null   object 
 11  Producers        5571 non-null   object 
 12  Studios          6863 non-null   object 
 13  Genres           7992 non-null   object 
 14  Themes           5360 non-null   object 
 15  Demographic      2703 non-null   object 
dtypes: float64(2), int64(3), object(11)
memory usage: 1.1+ MB


In [9]:
# Since we have some leftover null values lets replace it with '' 
anime=anime.fillna('')
print(anime.isnull().sum())

MAL ID             0
Title              0
Medium Type        0
Original Source    0
Status             0
Rating             0
Score              0
ScoredBy           0
Members            0
Favourite          0
Synopsis           0
Producers          0
Studios            0
Genres             0
Themes             0
Demographic        0
dtype: int64


In [10]:
data = json.load(open('/content/drive/MyDrive/Anime_Recommender/anime-offline-database.json'))

In [11]:
dataList = data['data']
dataList

[{'sources': ['https://anilist.co/anime/142051',
   'https://anime-planet.com/anime/raise-a-suilen-nvade-show',
   'https://myanimelist.net/anime/51478'],
  'title': '!NVADE SHOW!',
  'type': 'SPECIAL',
  'episodes': 1,
  'status': 'FINISHED',
  'animeSeason': {'season': 'FALL', 'year': 2020},
  'picture': 'https://cdn.myanimelist.net/images/anime/1930/122178.jpg',
  'thumbnail': 'https://cdn.myanimelist.net/images/anime/1930/122178t.jpg',
  'synonyms': ['!nvade Show!', 'RAISE A SUILEN: !NVADE SHOW!'],
  'relations': ['https://anilist.co/anime/101633',
   'https://myanimelist.net/anime/37869'],
  'tags': ['band', 'full cgi', 'music']},
 {'sources': ['https://anidb.net/anime/10143',
   'https://anilist.co/anime/102416',
   'https://anime-planet.com/anime/chiaki-kuriyama-0',
   'https://anisearch.com/anime/9010',
   'https://kitsu.io/anime/8925',
   'https://myanimelist.net/anime/20707',
   'https://notify.moe/anime/Ff1bpKmmR'],
  'title': '"0"',
  'type': 'SPECIAL',
  'episodes': 1,
  '

In [12]:
title = []
tags = []
status = []
media_type= []
sources = []


for i in range(len(dataList)):
  if dataList[i] is not None:
    tags.append(dataList[i]['tags'])
    title.append(dataList[i]['title'])
    sources.append(dataList[i]['sources'])
    media_type.append(dataList[i]['type'])
    status.append(dataList[i]['status'])



In [13]:
new_df = pd.DataFrame({'title': title, 'updated_tags': tags,'status':status,'media_type':media_type,'source':sources})

In [14]:
def extract_mal_id(urls):
    for url in urls:
        if 'myanimelist.net' in url:
            match = re.search(r'/anime/(\d+)', url)
            if match:
                return match.group(1)
    return None

In [15]:
new_df['status'].value_counts()

FINISHED    29349
UPCOMING      856
ONGOING       447
UNKNOWN        72
Name: status, dtype: int64

In [16]:
new_df['MAL ID'] = new_df['source'].apply(lambda x: extract_mal_id(x))
new_df['MAL ID']

0        51478
1        20707
2        25627
3        54496
4         7669
         ...  
30719     None
30720     None
30721     None
30722    31687
30723    54578
Name: MAL ID, Length: 30724, dtype: object

In [17]:
final_df = new_df[(new_df['MAL ID'].notna()) & (new_df['media_type'].isin(['TV','MOVIE','ONA'])) & (new_df['updated_tags'].notna()) & (new_df['status'].isin(['FINISHED','ONGOING'])) ]
final_df

,title,updated_tags,status,media_type,source,MAL ID
2,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...","[family friendly, fantasy, frogs, kids]",FINISHED,MOVIE,"[https://anidb.net/anime/3689, https://anime-p...",25627
3,"""Ai"" wo Taberu",[drama],FINISHED,MOVIE,[https://myanimelist.net/anime/54496],54496
6,"""Bungaku Shoujo"" Movie","[asia, based on a light novel, classic literat...",FINISHED,MOVIE,"[https://anidb.net/anime/6959, https://anilist...",6408
7,"""Calpis"" Hakkou Monogatari","[advertisement, food and beverage, historical,...",FINISHED,ONA,"[https://anilist.co/anime/138257, https://anim...",38045
8,"""Eiji""","[achromatic, anti-hero, based on a manga, boxi...",FINISHED,MOVIE,"[https://anidb.net/anime/4317, https://anilist...",6076
...,...,...,...,...,...,...
30701,Zuqiu Wangzhe,"[adventure, cg-anime, comedy, fantasy, ganbatt...",FINISHED,MOVIE,"[https://anisearch.com/anime/14438, https://my...",45069
30706,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,"[asia, based on a song, childhood friends, dra...",FINISHED,MOVIE,"[https://anidb.net/anime/11376, https://anilis...",31245
30716,Üks Uks,"[avant garde, dementia, shorts, stop motion an...",FINISHED,MOVIE,"[https://anime-planet.com/anime/uks-uks, https...",29708
30717,ēlDLIVE,"[action, aliens, based on a manga, body sharin...",FINISHED,TV,"[https://anidb.net/anime/11970, https://anilis...",32878


In [18]:
final_df['MAL ID'] = final_df['MAL ID'].astype('int64')

<ipython-input-18-4e944c64014a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['MAL ID'] = final_df['MAL ID'].astype('int64')


In [19]:
merged_df = anime.merge(final_df, on='MAL ID', how='inner')

In [20]:
merged_df

,MAL ID,Title,Medium Type,Original Source,Status,Rating,Score,ScoredBy,Members,Favourite,...,Producers,Studios,Genres,Themes,Demographic,title,updated_tags,status,media_type,source
0,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,Finished Airing,R - 17+ (violence & profanity),9.11,1972869.0,3103521,214047,...,"['Aniplex', 'Square Enix', 'Mainichi Broadcast...",['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy']",['Military'],['Shounen'],Fullmetal Alchemist: Brotherhood,"[action, adoption, adventure, alchemy, alterna...",FINISHED,TV,"[https://anidb.net/anime/6107, https://anilist..."
1,41467,Bleach: Sennen Kessen-hen,TV,Manga,Finished Airing,R - 17+ (violence & profanity),9.09,179829.0,399540,16156,...,"['TV Tokyo', 'Aniplex', 'Dentsu', 'Shueisha']",['Pierrot'],"['Action', 'Adventure', 'Fantasy']",,['Shounen'],Bleach: Sennen Kessen-hen,"[action, adventure, afterlife, angels, based o...",FINISHED,TV,"[https://anidb.net/anime/15449, https://anilis..."
2,9253,Steins;Gate,TV,Visual novel,Finished Airing,PG-13 - Teens 13 or older,9.08,1309377.0,2389026,180062,...,"['Frontier Works', 'Media Factory', 'Movic', '...",['White Fox'],"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",,Steins;Gate,"[alternate universe, amnesia, asia, based on a...",FINISHED,TV,"[https://anidb.net/anime/7729, https://anilist..."
3,28977,Gintama°,TV,Manga,Finished Airing,PG-13 - Teens 13 or older,9.07,232370.0,580694,15592,...,"['TV Tokyo', 'Aniplex', 'Dentsu']",['Bandai Namco Pictures'],"['Action', 'Comedy', 'Sci-Fi']","['Gag Humor', 'Historical', 'Parody', 'Samurai']",['Shounen'],Gintama°,"[action, adventure, alien, aliens, alternative...",FINISHED,TV,"[https://anidb.net/anime/10993, https://anilis..."
4,43608,Kaguya-sama wa Kokurasetai: Ultra Romantic,TV,Manga,Finished Airing,PG-13 - Teens 13 or older,9.07,412788.0,767095,27506,...,"['Aniplex', 'Mainichi Broadcasting System', 'M...",['A-1 Pictures'],"['Comedy', 'Romance']",['School'],['Seinen'],Kaguya-sama wa Kokurasetai: Ultra Romantic,"[based on a manga, cgi, comedy, coming of age,...",FINISHED,TV,"[https://anidb.net/anime/15806, https://anilis..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8423,32894,Love Bites,ONA,Original,Finished Airing,R - 17+ (violence & profanity),2.58,2889.0,4071,2,...,,,['Horror'],,,Love Bites,"[cannibalism, cg animation, cg-anime, demons, ...",FINISHED,ONA,"[https://anidb.net/anime/12288, https://anilis..."
8424,16608,Shitcom,ONA,Original,Finished Airing,PG-13 - Teens 13 or older,2.76,15484.0,20507,50,...,,,"['Comedy', 'Romance']",,,Shitcom,"[body horror, comedy, crude, cute girls doing ...",FINISHED,ONA,"[https://anidb.net/anime/10647, https://anilis..."
8425,22179,Aki no Puzzle,Movie,Original,Finished Airing,R+ - Mild Nudity,2.39,1526.0,2254,0,...,,,['Avant Garde'],,,Aki no Puzzle,"[abstract, avant garde, dementia, nudity, shor...",FINISHED,MOVIE,"[https://anidb.net/anime/11355, https://anilis..."
8426,38853,Ex-Arm,TV,Manga,Finished Airing,PG-13 - Teens 13 or older,2.91,41248.0,78006,294,...,"['Nippon Columbia', 'BS Fuji', 'Shueisha', 'Yo...",['Visual Flight'],"['Action', 'Sci-Fi']",,['Seinen'],Ex-Arm,"[action, aliens, amnesia, android, androids, a...",FINISHED,TV,"[https://anidb.net/anime/14545, https://anilis..."


In [21]:
merged_df = merged_df.dropna()

In [22]:
merged_df

,MAL ID,Title,Medium Type,Original Source,Status,Rating,Score,ScoredBy,Members,Favourite,...,Producers,Studios,Genres,Themes,Demographic,title,updated_tags,status,media_type,source
0,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,Finished Airing,R - 17+ (violence & profanity),9.11,1972869.0,3103521,214047,...,"['Aniplex', 'Square Enix', 'Mainichi Broadcast...",['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy']",['Military'],['Shounen'],Fullmetal Alchemist: Brotherhood,"[action, adoption, adventure, alchemy, alterna...",FINISHED,TV,"[https://anidb.net/anime/6107, https://anilist..."
1,41467,Bleach: Sennen Kessen-hen,TV,Manga,Finished Airing,R - 17+ (violence & profanity),9.09,179829.0,399540,16156,...,"['TV Tokyo', 'Aniplex', 'Dentsu', 'Shueisha']",['Pierrot'],"['Action', 'Adventure', 'Fantasy']",,['Shounen'],Bleach: Sennen Kessen-hen,"[action, adventure, afterlife, angels, based o...",FINISHED,TV,"[https://anidb.net/anime/15449, https://anilis..."
2,9253,Steins;Gate,TV,Visual novel,Finished Airing,PG-13 - Teens 13 or older,9.08,1309377.0,2389026,180062,...,"['Frontier Works', 'Media Factory', 'Movic', '...",['White Fox'],"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",,Steins;Gate,"[alternate universe, amnesia, asia, based on a...",FINISHED,TV,"[https://anidb.net/anime/7729, https://anilist..."
3,28977,Gintama°,TV,Manga,Finished Airing,PG-13 - Teens 13 or older,9.07,232370.0,580694,15592,...,"['TV Tokyo', 'Aniplex', 'Dentsu']",['Bandai Namco Pictures'],"['Action', 'Comedy', 'Sci-Fi']","['Gag Humor', 'Historical', 'Parody', 'Samurai']",['Shounen'],Gintama°,"[action, adventure, alien, aliens, alternative...",FINISHED,TV,"[https://anidb.net/anime/10993, https://anilis..."
4,43608,Kaguya-sama wa Kokurasetai: Ultra Romantic,TV,Manga,Finished Airing,PG-13 - Teens 13 or older,9.07,412788.0,767095,27506,...,"['Aniplex', 'Mainichi Broadcasting System', 'M...",['A-1 Pictures'],"['Comedy', 'Romance']",['School'],['Seinen'],Kaguya-sama wa Kokurasetai: Ultra Romantic,"[based on a manga, cgi, comedy, coming of age,...",FINISHED,TV,"[https://anidb.net/anime/15806, https://anilis..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8423,32894,Love Bites,ONA,Original,Finished Airing,R - 17+ (violence & profanity),2.58,2889.0,4071,2,...,,,['Horror'],,,Love Bites,"[cannibalism, cg animation, cg-anime, demons, ...",FINISHED,ONA,"[https://anidb.net/anime/12288, https://anilis..."
8424,16608,Shitcom,ONA,Original,Finished Airing,PG-13 - Teens 13 or older,2.76,15484.0,20507,50,...,,,"['Comedy', 'Romance']",,,Shitcom,"[body horror, comedy, crude, cute girls doing ...",FINISHED,ONA,"[https://anidb.net/anime/10647, https://anilis..."
8425,22179,Aki no Puzzle,Movie,Original,Finished Airing,R+ - Mild Nudity,2.39,1526.0,2254,0,...,,,['Avant Garde'],,,Aki no Puzzle,"[abstract, avant garde, dementia, nudity, shor...",FINISHED,MOVIE,"[https://anidb.net/anime/11355, https://anilis..."
8426,38853,Ex-Arm,TV,Manga,Finished Airing,PG-13 - Teens 13 or older,2.91,41248.0,78006,294,...,"['Nippon Columbia', 'BS Fuji', 'Shueisha', 'Yo...",['Visual Flight'],"['Action', 'Sci-Fi']",,['Seinen'],Ex-Arm,"[action, aliens, amnesia, android, androids, a...",FINISHED,TV,"[https://anidb.net/anime/14545, https://anilis..."


In [23]:
# Lets create a class to clean and tokenize our dirty corpus with help from the nltk library
class CleanCorpus:
  def __init__(self,df):
     self.df = df
     self.lemmatizer = WordNetLemmatizer()
     self.detokenizer = MosesDetokenizer()

  # This function will remove all non-alphabetic characters,newlines and whitespace as well as lowercase the corpus for further NLP cleaning 
  def remove_punctuation(self,text,lowercase=True):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'[/\n]', '', text)
    text = re.sub(r'\s+', ' ', text)
    if lowercase:
      text = text.lower()
    for char in string.punctuation:
      text = text.replace(char," ")
    return text

  def tokenize_text(self,text):
     return word_tokenize(text)

  def remove_stopwords(self,tokens,stopwords=set(stopwords.words("english"))):
    return [token for token in tokens if token not in stopwords]
  
  def get_wordnet_pos(self,tag):
      if tag.startswith('J'):
          return wordnet.ADJ
      elif tag.startswith('V'):
          return wordnet.VERB
      elif tag.startswith('N'):
          return wordnet.NOUN
      elif tag.startswith('R'):
          return wordnet.ADV
      else:
          return wordnet.NOUN
    
  def preprocess_text(self):
      self.df['remove_punctuation'] = self.df['corpus'].apply(lambda x:self.remove_punctuation(x))
      self.df['tokenize_text'] = self.df['remove_punctuation'].apply(self.tokenize_text)
      self.df['remove_stopwords'] = self.df['tokenize_text'].apply(self.remove_stopwords)

      # Apply pos_tag convert the tokens based on their context as the same word might be used as Verbs or Nouns.Thus this method
      # identify the correct sense of such words based on their syntactic context.
      self.df['applied_pos_tag'] = self.df['remove_stopwords'].apply(nltk.tag.pos_tag)
      
      # After applying the pos_tag the function get_wordnet_pos maps the part-of-speech tags returned by NLTK's pos_tag function to the corresponding WordNet
      # part-of-speech tags, which are used by the WordNetLemmatizer
      self.df['get_wordnet_pos'] = self.df['applied_pos_tag'].apply(lambda x: [(word, self.get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

      # Lets lemmatize the text
      self.df['lemmatized_text'] = self.df['get_wordnet_pos'].apply(lambda x: [self.lemmatizer.lemmatize(word, tag) for word, tag in x])

      # Lets turn the tokens back into sentences for the SBERT Training 
      self.df['detoken'] = self.df['lemmatized_text'].apply(lambda x:self.detokenizer.detokenize(x,return_str=True))
      self.df = self.df.drop(['remove_punctuation','tokenize_text','remove_stopwords','applied_pos_tag','get_wordnet_pos','lemmatized_text'],axis=1)

      return self.df

In [24]:
def model_save():
  model = SentenceTransformer('all-mpnet-base-v2')
  model.save('/content/drive/MyDrive/Anime_Recommender/model/test_model')

In [25]:
model_save()
model = SentenceTransformer('/content/drive/MyDrive/Anime_Recommender/model/test_model')

In [26]:
def train():
  descriptions = anime.detoken.values
  embeddings = []
  for i,des in enumerate(descriptions):
    embeddings.append(model.encode(des))
  f = gzip.GzipFile("/content/drive/MyDrive/Anime_Recommender/similar_title_embeddings.npy.gz", "w")
  np.save(file=f, arr=embeddings)
  f.close()

In [27]:
anime = merged_df

In [28]:
anime['corpus'] = anime['updated_tags'].astype('str') + ' '+anime['Studios']+ ' '+anime['Medium Type']

In [29]:
clean_text = CleanCorpus(anime)
anime = clean_text.preprocess_text()

In [30]:
anime['detoken'][55]

'action adventure animal archery asia award win body horror curse demon disability drama earth environmental fantasy female protagonist feudal japan forest god gore gun hero strong character heroine strong character historical japan japanese mythology kaiju kamis male protagonist mina mythology new old asia original work outdoor past revenge romance royalty rural samurai splatter struggle survive sword co tomboy travel tsundere war youkai studio ghibli movie'

In [31]:
train()


In [32]:
anime_df = anime
anime_df = anime_df.reset_index()
data = anime_df
indices = pd.Series(anime_df.index, index=anime_df['Title'])

In [33]:
# Saving the merged dataset for later deployement
data.to_csv('/content/drive/MyDrive/Anime_Recommender/finalanime.csv')

In [34]:
data= pd.read_csv('/content/drive/MyDrive/Anime_Recommender/finalanime.csv')

In [35]:
f = gzip.GzipFile("/content/drive/MyDrive/Anime_Recommender/similar_title_embeddings.npy.gz", "r")
embeddings_recommender = np.load(f)

In [36]:
# Now lets build a recommender system using the same embeddings but for anime_titles only 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cos_sim_data = pd.DataFrame(cosine_similarity(embeddings_recommender))

def similar_title(title,medium_type, N = 200000):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim_data[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:N+1]
    movie_indices = [i[0] for i in sim_scores]
    final_data = data.iloc[movie_indices]
    similarity_score = [x[1] for x in sim_scores]
    final_data['similarity_score'] = [round(x[1] * 100, 2) for x in sim_scores]
    print(sim_scores)
    m = round(data['ScoredBy'].quantile(0.95))
    C = round(data['Score'].mean(), 1)

    def weighted_rating(x):
      v = x['ScoredBy']
      R = x['Score']
      return round((v*R/(v+m)) + (m*C/(m+v)),1)
    final_data['weighted_rating'] = final_data.apply(weighted_rating,axis=1)
    
    if title in final_data['Title'].values:
      final_data =final_data[final_data['Title'] != title]

    del final_data['index']
    final_data = final_data.iloc[:,[0,1,2,3,6,8,9,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    final_data = final_data[final_data['Medium Type'].isin(medium_type)]
    return final_data

In [37]:
s = similar_title("Ex-Arm",['TV','Movie','ONA'],10)
s 

[(1629, 0.91654372215271), (170, 0.9131333827972412), (2892, 0.9131233096122742), (1050, 0.9099313020706177), (2086, 0.9052503108978271), (5457, 0.9006199836730957), (1656, 0.9004631638526917), (1392, 0.8995957970619202), (4760, 0.8985273838043213), (884, 0.896761417388916)]


<ipython-input-36-a12ee1dd4424>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['similarity_score'] = [round(x[1] * 100, 2) for x in sim_scores]
<ipython-input-36-a12ee1dd4424>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['weighted_rating'] = final_data.apply(weighted_rating,axis=1)


,Unnamed: 0,MAL ID,Title,Medium Type,Rating,ScoredBy,Members,Producers,Studios,Genres,Themes,Demographic,title,updated_tags,status,media_type,source,corpus,detoken,similarity_score
1629,1629,32071,Gantz:O,Movie,R+ - Mild Nudity,57041.0,93984,"['Dentsu', 'Toho', 'Lawson HMV Entertainment',...",['Digital Frontier'],"['Action', 'Drama', 'Horror', 'Sci-Fi', 'Super...","['Gore', 'Psychological', 'Survival']",['Seinen'],Gantz:O,"['action', 'action drama', 'adventure', 'alien...",FINISHED,MOVIE,"['https://anidb.net/anime/11778', 'https://ani...","['action', 'action drama', 'adventure', 'alien...",action action drama adventure alien base manga...,91.65
170,170,35557,Houseki no Kuni,TV,PG-13 - Teens 13 or older,181035.0,424059,"['Mainichi Broadcasting System', 'Kodansha', '...",['Orange'],"['Action', 'Drama', 'Fantasy', 'Mystery']",['Anthropomorphic'],['Seinen'],Houseki no Kuni,"['action', 'agender', 'alternate universe', 'a...",FINISHED,TV,"['https://anidb.net/anime/13165', 'https://ani...","['action', 'agender', 'alternate universe', 'a...",action agender alternate universe alternative ...,91.31
2892,2892,32086,Blame! Movie,Movie,PG-13 - Teens 13 or older,56480.0,101850,"['Starchild Records', 'Kodansha', 'KlockWorx',...",['Polygon Pictures'],"['Action', 'Drama', 'Sci-Fi']","['Mecha', 'Psychological']",['Seinen'],Blame! Movie,"['action', 'action drama', 'androids', 'apocal...",FINISHED,MOVIE,"['https://anidb.net/anime/11782', 'https://ani...","['action', 'action drama', 'androids', 'apocal...",action action drama androids apocalyptic ficti...,91.31
1050,1050,19775,Sidonia no Kishi,TV,R+ - Mild Nudity,106918.0,222188,"['Starchild Records', 'Mainichi Broadcasting S...",['Polygon Pictures'],"['Action', 'Sci-Fi']","['Mecha', 'Space']",['Seinen'],Sidonia no Kishi,"['action', 'adventure', 'agender', 'alien', 'a...",FINISHED,TV,"['https://anidb.net/anime/9892', 'https://anil...","['action', 'adventure', 'agender', 'alien', 'a...",action adventure agender alien alien animeism ...,90.99
2086,2086,4618,RideBack,TV,PG-13 - Teens 13 or older,20633.0,46580,['Geneon Universal Entertainment'],['Madhouse'],"['Action', 'Drama', 'Sci-Fi']","['Mecha', 'School']",['Seinen'],RideBack,"['action', 'based on a manga', 'cgi', 'coming ...",FINISHED,TV,"['https://anidb.net/anime/6235', 'https://anil...","['action', 'based on a manga', 'cgi', 'coming ...",action base manga cgi come age cyberpunk dance...,90.53
5457,5457,38749,Blade Runner: Black Lotus,TV,R - 17+ (violence & profanity),4127.0,23242,NaN,['Sola Digital Arts'],"['Sci-Fi', 'Suspense']",NaN,NaN,Blade Runner: Black Lotus,"['action', 'based on a movie', 'cg animation',...",FINISHED,TV,"['https://anidb.net/anime/14511', 'https://ani...","['action', 'based on a movie', 'cg animation',...",action base movie cg animation cganime cyberpu...,90.06
1656,1656,31580,Ajin,TV,R - 17+ (violence & profanity),283167.0,562383,"['Mainichi Broadcasting System', 'Kodansha', '...",['Polygon Pictures'],"['Action', 'Horror', 'Mystery', 'Supernatural']",['Gore'],['Seinen'],Ajin,"['action', 'animeism', 'anti-hero', 'based on ...",FINISHED,TV,"['https://anidb.net/anime/11577', 'https://ani...","['action', 'animeism', 'anti-hero', 'based on ...",action animeism antihero base manga body horro...,90.05
1392,1392,12487,Kyoukaisenjou no Horizon II,TV,R+ - Mild Nudity,36058.0,80562,"['Bandai Visual', 'Lantis', 'ASCII Media Works...",['Sunrise'],"['Action', 'Fantasy', 'Sci-Fi']",NaN,NaN,Kyoukaisenjou no Horizon II,"['action', 'action comedy', 'adventure', 'alte...",FINISHED,TV,"['https://anidb.net/anime/8886', 'https://anil...","['action', 'action comedy', 'adventure', 'alte...",action action comedy adventure alternative wor...,89.96
4760,4760,38663,Mushikago no Cagaster,ONA,R - 17+ (violence & profanity),15443.0,35655,NaN,['Studio Kai'],"['Action', 'Drama', 'Sci-Fi']",NaN,NaN,Mushikago no Cagaster,"['action', 'adventure', 'age gap', 'amnesia', ...",FINISHED,ONA,"['https://anidb.net/anime/14479', 'https://ani...","['action', 'adventu